# import

In [1]:
import pymysql
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import pymysql
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [2]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

/Users/yunju/miniforge3/envs/news/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from openai import OpenAI
import json
from collections import OrderedDict

import pickle
from sklearn.feature_selection import RFE
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import openai
import os

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumBarunGothic'

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain


In [4]:
from konlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from gensim.models import doc2vec

In [5]:
def create_db(passwd, db, host='127.0.0.1', port=3306, user='root'):
    conn = pymysql.connect(
            host=host,
            port=port,
            user=user,
            passwd=passwd,
            db=db,
            charset='utf8'
        )
    cursur = conn.cursor()
    return conn, cursur

# cluster

In [6]:
conn, cursor = create_db('password', 'newspeed')

In [7]:
# User-Agent 설정

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

# 카테고리별 URL 및 class 명 설정
category_info = [
    {'category': 'politics', 'url': 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100', 'headline_class': 'sh_text_headline nclicks(cls_pol.clsart)', 'related_url_class': 'sh_head_more nclicks(cls_pol.clstitle)'},
    {'category': 'economy', 'url': 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101', 'headline_class': 'sh_text_headline nclicks(cls_eco.clsart)', 'related_url_class': 'sh_head_more nclicks(cls_eco.clstitle)'},
    {'category': 'society', 'url': 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102', 'headline_class': 'sh_text_headline nclicks(cls_nav.clsart)', 'related_url_class': 'sh_head_more nclicks(cls_nav.clstitle)'},
    {'category': 'culture', 'url': 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103', 'headline_class': 'sh_text_headline nclicks(cls_lif.clsart)', 'related_url_class': 'sh_head_more nclicks(cls_lif.clstitle)'},
    {'category': 'science', 'url': 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105', 'headline_class': 'sh_text_headline nclicks(cls_sci.clsart)', 'related_url_class': 'sh_head_more nclicks(cls_sci.clstitle)'},
    {'category': 'world', 'url': 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=104', 'headline_class': 'sh_text_headline nclicks(cls_wor.clsart)', 'related_url_class': 'sh_head_more nclicks(cls_wor.clstitle)'},
]


# 각 카테고리별로 DataFrame을 생성하고 리스트에 추가
dataframes = []

cursor.execute("SELECT MAX(cid) FROM cluster")
max_cid = cursor.fetchone()[0]

# 만약 테이블에 데이터가 없다면 max_cid는 None일 수 있으므로 기본값을 0으로 설정
max_cid = max_cid if max_cid is not None else 0

In [8]:
# frist_cid = 151
first_cid = max_cid

In [9]:
first_cid = 0

In [11]:


 # 각 컬럼의 데이터를 바로 데이터베이스에 삽입
insert_query = """
    INSERT IGNORE INTO cluster (cid, title, category, clu_url, num)
    VALUES (%s, %s, %s, %s, %s)
"""


clu_id = []

for category_info_item in category_info:
    category = category_info_item['category']
    url = category_info_item['url']
    headline_class = category_info_item['headline_class']
    related_url_class = category_info_item['related_url_class']
    

    # 카테고리별로 10번 새로고침 수행
    for _ in range(10):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 필요한 데이터를 저장할 빈 리스트 초기화
        urls = []
        titles = []
        related_urls = []
        related_counts = []
        cid = []

        # class가 'sh_text'인 모든 요소를 찾기
        sh_text_elements = soup.find_all(class_='sh_text')

        for element in sh_text_elements:
            # class가 'sh_head_more nclicks(cls_pol.clstitle)'인 요소에서 관련 기사 URL 추가
            related_urls_element = element.find(class_=related_url_class)

            if related_urls_element:
                value = None
                # 추출된 URL 앞에 'https://news.naver.com' 추가
                full_url = 'https://news.naver.com' + related_urls_element['href']
                match = re.search(r'id=c_(\d+_\d+)', full_url)

                if match:
                    value = match.group(1)
                # match값이 있으면 for문 처음으로 돌아가고,
                # 없으면 리스트의 길이(새 뉴스 ID)를 사용하며, URL을 리스트에 추가
                if value in clu_id:
                    continue
                else:
                    # cid 추가
                    next_cid = max_cid + 1
                    cid.append(next_cid)
                    max_cid = next_cid
                    # clu_url 추가
                    related_urls.append(full_url)
                    clu_id.append(value)

                    # titles 추가
                    headline_element = element.find(class_=headline_class)
                    if headline_element:
                        titles.append(headline_element.text)
                    else:
                        titles.append(None)

                    # related_counts 추가
                    related_counts_element = element.find(class_='sh_head_more_icon_num')
                    if related_counts_element:
                        related_counts.append(related_counts_element.text)
                    else:
                        related_counts.append(None)

            
            else:
                # 관련 헤드라인 주소가 없다면 그냥 넘어가도록
                continue
           
            cursor.execute(insert_query, (cid[-1], titles[-1], category, related_urls[-1], related_counts[-1]))


        conn.commit()  # 변경사항 커밋
        # 데이터를 Pandas DataFrame으로 저장
        data = {
            'cid': cid,
            'title': titles,
            'category': category,
            'clu_url': related_urls,
            'num': related_counts
        }

        df = pd.DataFrame(data)
        dataframes.append(df)

# 모든 데이터프레임을 하나로 합치기
final_df = pd.concat(dataframes, ignore_index=True)


# 연결 종료
cursor.close()
conn.close()

In [25]:
sh_text_elements = soup.find_all(class_='sh_text')
sh_text_elements

[]

# news

In [12]:
conn, cursor = create_db('password', 'newspeed')

In [13]:
# 시간 처리 함수
def format_datetime(ymd_str):
    date_parts = ymd_str.split('.')[0:3]
    date_format = '-'.join(date_parts)

    time_match = re.search(r"(\d+:\d+)", ymd_str)
    if time_match:
        time_format = time_match.group(1)

        if "오후" in ymd_str and not time_format.startswith('12'):  # 오후이지만 12시가 아닌 경우
            hour = int(time_format.split(':')[0]) + 12  # 오후는 시간에 12를 더함
            time_format = f"{hour}:{time_format.split(':')[1]}"
        elif "오전" in ymd_str and time_format.startswith('12'):  # 오전 12시인 경우
            time_format = f"00:{time_format.split(':')[1]}"

        return f"{date_format} {time_format}:00"

    return f"{date_format} 00:00:00"

# 댓글 갯수 가져오는 함수
def extract_comment(article_url):
    pattern = re.compile(r'/(\d+)/(\d+)')
    match = pattern.search(article_url)

    if match:
        oid = match.group(1)
        aid = match.group(2)
    else:
        return int(0) # 일치되는게 없으면 댓글개수 0으로
    
    headers = {
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)',
    'referer' : article_url
    }

    url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&pool=cbox5&lang=ko&country=KR&objectId=news{}%2C{}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1"
    req_url = url.format(oid, aid)

    response = requests.get(req_url, headers=headers)
    
    # json 파일 출력
    test_text = response.text
    test_text = test_text.replace("_callback(","")[:-2]
    comment_text = json.loads(test_text)

    return int(comment_text['result']['count']['comment'])



In [14]:
# 단일 뉴스 기사 정보 가져오는 함수
def text_crawl(url, headers):
    article_info = {}

    response = requests.get(url, headers=headers)
    
    # 요청 성공
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 기사 제목
        title_element = soup.find('div', class_='media_end_head_title')
        article_title = title_element.get_text().strip() if title_element else ""

        # 기사 본문
        article_content_element = soup.find('article', class_='go_trans _article_content')
        if article_content_element:
            # 특정 부분을 제외한 모든 텍스트 요소 가져오기
            for tag in article_content_element.find_all(['b','div', 'span', 'em', 'br', 'img']):
                tag.extract()  # 특정 태그들 제거
            
            # 기사 본문 텍스트 추출
            article_text = article_content_element.get_text(strip=True)
        else:
            article_text = ""


        # 날짜
        article_date_1_element = soup.find(class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME")
        article_date_1 = article_date_1_element.get_text().strip() if article_date_1_element else ""

        # 날짜 정보 변경 (date형에 맞게끔)
        if article_date_1:
            article_date_1 = format_datetime(article_date_1)
        

        # 기자 메일/기자명
        journalist_info_element = soup.find(class_="byline_s")
        journalist_info = journalist_info_element.get_text().strip() if journalist_info_element else ""

        # 댓글 추출
        comment_count = extract_comment(url)

        # 이미지 추출
        img_url = None
        meta_tag = soup.find('meta', {'property': 'og:image'})
        if meta_tag:
            img_url = meta_tag.get('content')

        # 기사 정보를 딕셔너리로 저장합니다.
        article_info = {
            'title': article_title,
            'content': article_text,
            'ymd': article_date_1,
            'com_num' : comment_count,
            'j_info': journalist_info,
            'url': url,
            'img_url' : img_url
        }

    
    else:
        print(f"HTTP 요청 실패  {response.status_code}")
    

    
    return article_info

In [15]:
first_cid, max_cid

(0, 0)

In [16]:

# 데이터 삽입을 위한 SQL 쿼리
sql_insert = """
INSERT IGNORE INTO news (cid, category, title, content, ymd, com_num, j_info, company, url, img_url) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""



# 각 카테고리에 해당하는 클래스명을 딕셔너리로 정의합니다.
category_class_mapping = {
    'politics': 'cls_pol',
    'economy': 'cls_eco',
    'society': 'cls_nav',
    'culture': 'cls_lif',
    'science': 'cls_sci',
    'world': 'cls_wor'
}

sql_select_max_cid = "SELECT MAX(cid) FROM cluster"
cursor.execute(sql_select_max_cid)
max_cid = cursor.fetchone()[0]


sql_select_filtered = "SELECT clu_url, category, cid FROM cluster WHERE cid >= %s AND cid <= %s"
cursor.execute(sql_select_filtered, (first_cid, max_cid))
rows = cursor.fetchall()


cursor.close()
conn.close()

all_article_data = []  # 모든 기사 데이터를 저장할 리스트


batch_size = 100  # 한 번에 데이터베이스에 삽입할 행의 수

In [17]:
conn, cursor = create_db('password', 'newspeed')

In [19]:
rows

()

In [18]:

for row in tqdm(rows):
    cluster_url = row[0]
    category = row[1]
    cid = row[2]

    
    response = requests.get(cluster_url, headers=headers)


    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # 관련 기사 제목 및 내용 가져오기
        try:
            second_div = soup.find_all('div', class_='cluster')[1]
        except IndexError:
            print("next")
        
        # 관련 기사 목록
        li_tags = second_div.find_all('li')
        for li_tag in li_tags:
            category_class = category_class_mapping.get(category, '')  # 카테고리에 해당하는 클래스명 가져오기
            a_tag = li_tag.find('a', class_='nclicks({}.clsart1)'.format(category_class))
            # 신문사
            writing_tag = li_tag.find('span', class_='writing')
            company = writing_tag.get_text().strip() if writing_tag else ""
            # 기사 url
            url = a_tag['href'] if a_tag else ""
            
            # url 정보에 따른 기사 딕셔너리
            article = text_crawl(url, headers)

            article_data = (
                cid,
                category,
                article.get('title'),
                article.get('content'),
                article.get('ymd'),
                article.get('com_num'),
                article.get('j_info'),
                company,
                article.get('url'),
                article.get('img_url')
            )
        
            all_article_data.append(article_data)

        if len(all_article_data) >= batch_size:
            try:
                cursor.executemany(sql_insert, all_article_data)
                conn.commit()
                all_article_data = []
            except Exception as e:
                print(f"Error inserting data: {str(e)}")
                all_article_data = []
                conn.rollback()  # 오류 시 롤백

    else:
        print(f"HTTP 요청 실패 ({response.status_code})")


# 남아 있는 데이터를 데이터베이스에 삽입
if all_article_data:
    try:
        cursor.executemany(sql_insert, all_article_data)
        conn.commit()  # 변경사항 저장
    except Exception as e:
        print(f"Error inserting data: {str(e)}")
        conn.rollback()  # 오류 시 롤백


# 데이터베이스 연결 닫기
cursor.close()
conn.close()




0it [00:00, ?it/s]


# summary

In [27]:
# 데이터베이스 연결
conn, cursor = create_db('password', 'test')

# content가 빈 값인 레코드를 삭제하는 SQL 쿼리
sql_delete = "DELETE FROM news WHERE content = '' OR content IS NULL;"

# SQL 쿼리 실행
cursor.execute(sql_delete)

# 변경사항을 데이터베이스에 반영
conn.commit()

# 데이터베이스 연결 종료
cursor.close()
conn.close()

## 버블 키워드

In [28]:
conn, cursor = create_db('password', 'test')

In [29]:
sql_select_filtered = "SELECT cid, category, title, content, com_num FROM news WHERE cid >= %s AND cid <= %s"
cursor.execute(sql_select_filtered, (first_cid, max_cid))
news_data = cursor.fetchall()

conn.close()
cursor.close()

In [30]:
# 데이터를 Pandas DataFrame으로 변환
news_data = pd.DataFrame(news_data, columns=["cid", "category", "title", "content","com_num"])

# 전처리
news_data.title = news_data['title'].apply(lambda x: re.sub(r'\'|\\|\"|\[.*?\]|\(.*?\)', '', x.replace('"', '').replace('\'', '').replace('“', '').replace('”', '').replace('’', '').replace('‘', '').replace('···', ' ').replace('…', ' ').replace('...', ' ')).strip())

# 헤드라인 추출
headline_news = news_data.loc[news_data.groupby('cid')['com_num'].idxmax()].reset_index(drop=True)

# 클러스터 통합 댓글 개수
headline_news['sum_com'] = news_data.groupby('cid')['com_num'].sum().reset_index(drop=True)

# 관련기사 제목 4개 통합
grouped_data = news_data.groupby('cid')['title'].agg(lambda x: ' '.join(x.head(4))).reset_index()

In [31]:
os.environ["OPENAI_API_KEY"]='key'
openai.api_key = 'key'

In [32]:
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# Create an empty column for the summarized titles
grouped_data['summarized_title'] = ""

# Set the batch size
batch_size = 5

# Iterate over batches
for i in range(0, len(grouped_data), batch_size):
    # Get a batch of rows
    batch = grouped_data.iloc[i:i+batch_size]

    # Iterate over rows in the batch
    for j, (_, row) in enumerate(batch.iterrows()):
        # Create a prompt for each row in the batch 이것도 너무 길다... 3단어로 해야겟다
        prompt = (
            f"{j+1}. 이 기사의 제목을 한국어로 3단어 미만의 짧은 구나 문구로 요약하십시오. "
            "요약은 중요한 키워드를 포함해야 하며, 불필요한 단어는 제외하십시오. "
            "기사의 주요 내용을 간결하게 표현해야 합니다.\n\n"
            f"'{row['title']}'"
            "결과는 빈칸이 아니어야 합니다."
        )

        # Request completion from OpenAI for the individual prompt
        completion = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=100,
            temperature=0
        )

        # Extract and store the generated title
        generated_title = completion.choices[0].text.strip()
        print(f"{row.name + 1}. '{generated_title}'")
        grouped_data.at[row.name, 'summarized_title'] = generated_title

1. '여야, 652.7조 예산안 합의, R&D 예산 증액'
2. 'K-방산 수출 130억 달러 돌파, 세계 톱10 진입'
3. ''음주운전 논란, 하태경 장관과 이재명 의원의 대조적 대우''
4. ''이재명-김부겸 회동, 단합 위한 노력''
5. ''이재명 대통령 후보자, 자녀 용돈으로 1억 지급''
6. '"국민 4명 중 1명, 직장 내 갑질 경험 79%"'
7. '"인천 부평 캠프마켓, 84년만에 시민 품으로 완전 반환"'
8. ''여자 보복운전 논란''
9. ''부패 공직자 14명, 불법 재취업 적발''
10. '민주당, 김건희 특검 협의'
11. '조태열, 한중관계, 중요'
12. '민주당, 송영길 구속에 부패 의혹 곤혹'
13. '"한국, 2028년 세계 스포츠 강국으로 발전"'
14. '윤재옥 예산안 처리 후 비대위원장 결정, 국민의힘 의견 수렴 종료'
15. '"국가안보실, 행정전산망 사이버 위협 대비 강화"'
16. ''조정훈, 친족 강간 변호 논란에 대한 인재영입 공지연의 조치''
17. '"박민식, 서울현충원 세계 최고 추모 공간으로 조성"'
18. ''홍익표 조태용 부적격 장관, 총선용 개각 필요''
19. ''한동훈 비대위 결론 전망, 이재명-김부겸 통합 모색''
20. ''국회, 개식용 종식법 통과''
21. ''임종석 86세대, 새 시대로 가는 문 역할''
22. '윤재옥, 원로들 만나 위기상황 지도 요청'
23. '원희룡 국민의힘 비대위원장, 국민 마음 회복 위해 과감히 변해야'
24. '민형배 국힘, 불임정당 삭제 논란'
25. ''박민식 보훈 장관, 내년 총선 출마는 당에 백지 위임''
26. '한동훈, 말 너무 많이 마음 독해졌다? 특검법 말 아끼기'
27. ''김종인 한동훈 대선 출마 속도 놀랍고 대통령 후보 가능성''
28. '한동훈 비대위 출범 카운트다운'
29. '"작년 육아휴직 아빠 1명 중 70%는 대기업 다니며 2022년 육아휴직자 20만명 육박"'
30. ''내년 단독주택 공시가 0.57% 상승, 9년째 1위 집

In [33]:
grouped_data['summarized_title'] = grouped_data['summarized_title'].str.replace(r"[.\n'\"']", "", regex=True)
grouped_data

,cid,title,summarized_title
0,1,"여야, 652.7조 예산안 합의 정부안 대비 4.2조 감액 여야, 652.7조 예산...","여야, 6527조 예산안 합의, R&D 예산 증액"
1,2,"K-방산 대장주, 결국 수출이 생명줄 K-방산 실적 2년 연속 세계 톱10 올해 1...","K-방산 수출 130억 달러 돌파, 세계 톱10 진입"
2,3,국회의원은 음주운전 되고 장관은 안 되나 이재명 때린 하태경 음주운전 강도형 논란 ...,"음주운전 논란, 하태경 장관과 이재명 의원의 대조적 대우"
3,4,"이재명-김부겸 오찬 회동 통합 한목소리 김부겸, 정태근과 회동 제3지대 모색? 김부...","이재명-김부겸 회동, 단합 위한 노력"
4,5,이재명대통령 국민을 지배하는 왕인듯 농림부장관 후보자 제정신 아냐 이재명 자녀 용돈...,"이재명 대통령 후보자, 자녀 용돈으로 1억 지급"
...,...,...,...
128,129,"日경찰, 조총련 산하 무역상사 압수수색 北 부정송금 조사 일본 경찰, 조총련 산하 ...","일본 경찰, 조총련 산하 무역회사 압수수색으로 북한 유입 수입금 조사"
129,130,푸틴 국익에 따라 우크라전 협상 가능 핵전력 등 군 강화 절망의 우크라 전쟁 젤렌스...,"푸틴, 우크라 전쟁 계속 진행하며 핵무기 현대화 완료"
130,131,"日정부, 패트리엇 美수출 조율 방위장비 이전 원칙 개정 후 첫 사례 日, 美에 패트...","일본, 미국에 패트리엇 첫 수출 조율"
131,132,"가자 병원장, 의료진 중 하마스 대원 잠입 시인 하마스, 병원 통해 테러 활동 가자...","이스라엘 병원장, 하마스 대원 잠입 인정"


## 본문 요약

In [34]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS(GPU)를 사용합니다.")
else:
    device = torch.device("cpu")
    print("MPS(GPU)를 사용할 수 없으므로 CPU를 사용합니다.")


MPS(GPU)를 사용합니다.


In [35]:
# Load Model and Tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [36]:
model = model.to(device)

In [37]:

# 요약 생성 및 저장
max_input_length = 2048  # 입력 문장의 최대 길이 설정
max_summary_length = 300  # 요약의 최대 길이 설정 ( 토큰수 필요시 더 줄일 것 )

def generate_summary(row):
    input_text = row['content'][:max_input_length]  # 입력 문장을 최대 길이에 맞게 자름
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True).to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    summary_text_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        pad_token_id=model.config.pad_token_id,
        length_penalty=2.0,
        max_length=max_summary_length,
        min_length=56,
        num_beams=6,
    )
    summary = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    return summary


headline_news['summary'] = headline_news.apply(generate_summary, axis=1)
headline_news
# dd['summary'] = dd.apply(generate_summary, axis=1)
# dd


,cid,category,title,content,com_num,sum_com,summary
0,1,politics,여야 내년 예산안 합의 R&D 예산 6000억원 늘려,여야가 20일 내년도 예산안 처리에 합의했다. 예산안 법정 시한(2일)을 넘긴 지 ...,65,597,20일 예산안 법정 시한을 넘긴 지 19일 만에 여야는 국회 본회의를 열어 예산안을...
1,2,politics,올해 방산수출 130억 달러 웃돌아 2년 연속 세계 톱10,"수출국·무기체계 다변화…국방부 ""폴란드는 가장 중요한 파트너""(서울=연합뉴스) 박수...",8,18,"국방부는 올해 방위산업 수출 대상국 (폴란드, 아랍에미리트, UAE, 사우디아라비아..."
2,3,politics,하태경 음주운전 장관 안 된다? 이재명도 같은 해 음주운전,하태경 국민의힘 의원은 더불어민주당이 강도형 해수부장관 후보자의 음주운전 전과 등을...,840,2668,하태경 국민의힘 의원은 더불어민주당이 강도형 해수부장관 후보자의 음주운전 전과 등을...
3,4,politics,김부겸 범민주·진보 아울러야 이재명 힘 모으고 변화해야,(서울=연합뉴스) 홍지인 기자 = 더불어민주당 이재명 대표와 김부겸 전 국무총리가 ...,337,1390,더불어민주당 이재명 대표와 김부겸 전 국무총리가 내년 총선 승리를 위한 당 통합 방...
4,5,politics,이재명 1억을 자녀 용돈으로 줬다? 국민 공복 기본자세 안 돼,이재명 더불어민주당 대표는 송미령 농림축산식품부 장관 후보자의 인사청문회에서 제기된...,1193,6870,이재명 더불어민주당 대표는 19일 송미령 농림축산식품부 장관 후보자의 인사청문회에서...
...,...,...,...,...,...,...,...
128,129,world,"日경찰, 조총련 산하 무역상사 압수수색 北 부정송금 조사","조총련 산하 도카이상사, 中 기업 통해 북한에 송금 의혹[서울=뉴시스] 박준호 기자...",0,0,경시청 공안부가 중국 기업에 수입대금을 부정 송금한 혐의로 도카이상사와 도메이상사 ...
129,130,world,푸틴 내년에도 우크라 작전 계속 젤렌스키 50만명 징병 필요,블라디미르 푸틴 러시아 대통령은 19일(현지시간) “‘특별군사작전’ 목표를 포기할 ...,56,344,러시아라디미르 푸틴 러시아 대통령은 19일 “‘특별군사작전’ 목표를 포기할 생각이 ...
130,131,world,"日, 美에 패트리엇 첫수출 조율 우크라지원 미사일 부족 해소","이르면 22일 방위장비 이전 3원칙 등 개정…라이선스 완제품 수출 검토젤렌스키 ""키...",9,13,일본일 정부는 우크라이나 지원에 따른 미국의 미사일 부족 문제를 해소해 인도·태평양...
131,132,world,"가자 병원장, 의료진 중 하마스 대원 잠입 시인","이스라엘군·신베트 심문서 시인…""병원을 작전 기지로 활용""(카이로=연합뉴스) 김상훈...",22,51,이스라엘군은 19일 소셜미디어 엑스(X)를 통해 가자지구 북부 자발리야에 있는 카말...


In [38]:
summary_df=pd.merge(headline_news[['cid', 'category', 'summary', 'sum_com']],grouped_data[['cid', 'summarized_title']], on='cid')

summary_df

,cid,category,summary,sum_com,summarized_title
0,1,politics,20일 예산안 법정 시한을 넘긴 지 19일 만에 여야는 국회 본회의를 열어 예산안을...,597,"여야, 6527조 예산안 합의, R&D 예산 증액"
1,2,politics,"국방부는 올해 방위산업 수출 대상국 (폴란드, 아랍에미리트, UAE, 사우디아라비아...",18,"K-방산 수출 130억 달러 돌파, 세계 톱10 진입"
2,3,politics,하태경 국민의힘 의원은 더불어민주당이 강도형 해수부장관 후보자의 음주운전 전과 등을...,2668,"음주운전 논란, 하태경 장관과 이재명 의원의 대조적 대우"
3,4,politics,더불어민주당 이재명 대표와 김부겸 전 국무총리가 내년 총선 승리를 위한 당 통합 방...,1390,"이재명-김부겸 회동, 단합 위한 노력"
4,5,politics,이재명 더불어민주당 대표는 19일 송미령 농림축산식품부 장관 후보자의 인사청문회에서...,6870,"이재명 대통령 후보자, 자녀 용돈으로 1억 지급"
...,...,...,...,...,...
128,129,world,경시청 공안부가 중국 기업에 수입대금을 부정 송금한 혐의로 도카이상사와 도메이상사 ...,0,"일본 경찰, 조총련 산하 무역회사 압수수색으로 북한 유입 수입금 조사"
129,130,world,러시아라디미르 푸틴 러시아 대통령은 19일 “‘특별군사작전’ 목표를 포기할 생각이 ...,344,"푸틴, 우크라 전쟁 계속 진행하며 핵무기 현대화 완료"
130,131,world,일본일 정부는 우크라이나 지원에 따른 미국의 미사일 부족 문제를 해소해 인도·태평양...,13,"일본, 미국에 패트리엇 첫 수출 조율"
131,132,world,이스라엘군은 19일 소셜미디어 엑스(X)를 통해 가자지구 북부 자발리야에 있는 카말...,51,"이스라엘 병원장, 하마스 대원 잠입 인정"


In [39]:
conn, cursor = create_db('password', 'test')

In [40]:
# 데이터를 삽입할 SQL 쿼리
sql_insert = """
INSERT IGNORE INTO summary (cid, category, s_state, sum_com, keyword) 
VALUES (%s, %s, %s, %s, %s);
"""

try:
    # 데이터프레임의 각 행을 튜플로 변환하고 리스트에 추가
    data_to_insert = [(row.cid, row.category, row.summary, row.sum_com, row.summarized_title) for row in summary_df.itertuples(index=False)]

    # executemany를 사용하여 여러 행의 데이터를 한 번에 삽입
    cursor.executemany(sql_insert, data_to_insert)
    conn.commit()  # 변경사항을 커밋

except Exception as e:
    print(f"Error inserting data: {str(e)}")
    conn.rollback()  # 오류 발생 시 롤백

finally:
    # 데이터베이스 연결 종료
    cursor.close()
    conn.close()


# doc2vec 모델 훈련 및 저장

In [41]:
conn, cursor = create_db('password', 'test')


In [42]:
sql_select = """
SELECT content, cid
FROM (
    SELECT
        content,
        cid,
        com_num,
        ROW_NUMBER() OVER (PARTITION BY cid ORDER BY com_num DESC) as rn
    FROM news
) AS ranked_news
WHERE rn = 1;
"""

cursor.execute(sql_select)
rows = cursor.fetchall()
cursor.close()
conn.close()

df = pd.DataFrame(rows)
df.columns=['content','cid']

In [43]:
df

,content,cid
0,여야가 20일 내년도 예산안 처리에 합의했다. 예산안 법정 시한(2일)을 넘긴 지 ...,1
1,"수출국·무기체계 다변화…국방부 ""폴란드는 가장 중요한 파트너""(서울=연합뉴스) 박수...",2
2,하태경 국민의힘 의원은 더불어민주당이 강도형 해수부장관 후보자의 음주운전 전과 등을...,3
3,(서울=연합뉴스) 홍지인 기자 = 더불어민주당 이재명 대표와 김부겸 전 국무총리가 ...,4
4,이재명 더불어민주당 대표는 송미령 농림축산식품부 장관 후보자의 인사청문회에서 제기된...,5
...,...,...
128,"조총련 산하 도카이상사, 中 기업 통해 북한에 송금 의혹[서울=뉴시스] 박준호 기자...",129
129,블라디미르 푸틴 러시아 대통령은 19일(현지시간) “‘특별군사작전’ 목표를 포기할 ...,130
130,"이르면 22일 방위장비 이전 3원칙 등 개정…라이선스 완제품 수출 검토젤렌스키 ""키...",131
131,"이스라엘군·신베트 심문서 시인…""병원을 작전 기지로 활용""(카이로=연합뉴스) 김상훈...",132


In [44]:
mecab = Mecab("/opt/homebrew/lib/mecab/dic/mecab-ko-dic")
tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['content']
  tag = row['cid']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))
    
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.dv.index_to_key)}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)


100%|██████████| 133/133 [00:00<00:00, 391.01it/s]


Tag Size: 134 / 

In [45]:
# 모델 저장
model.save("../doc2vec_model_test")

In [ ]:
# 저장된 모델 불러오기
# loaded_model = Doc2Vec.load("../doc2vec_model")